In [87]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
writer_genres = pd.read_csv("../clean data/writer_genres.csv", index_col=0)
writer_genres.head()

,Writer Name,Genres
0,Nicaless,"Fantasy, Young Adult"
1,Rachel B. Moore,"Literary, Literary Fiction"
2,abookishbabe,Young Adult
3,alexabexis,"Romance, Horror/Supernatural, Horror & Superna..."
4,AllYellowFlowers,"Literary, Literary Fiction"


Below defines a function that calculates the jaccard distance from two different lists of genres.

In [89]:
def jaccard(a, b):
    if (type(a) == "float" or type(b) == float):
        return 0
    a = set(a.split(", "))
    b = set(b.split(", "))
    intersect = a.intersection(b)
    union = a.union(b)
    return float(len(intersect)) / len(union)

nicaless_genres = writer_genres['Genres'][writer_genres['Writer Name'] == "Nicaless"].values[0]
abookishbabe_genres = writer_genres['Genres'][writer_genres['Writer Name'] == "abookishbabe"].values[0]
jaccard(nicaless_genres, abookishbabe_genres)

0.5

Here defines a function that uses the jaccard function to calculate the distance between a given writer's list of genres and all other writers' genres and returns a set of suggested genres based on the top ten closes writers.

In [92]:
def getSimilar(writer):
    my_genres = writer_genres['Genres'][writer_genres['Writer Name'] == writer].values[0]
    
    writers = []
    genres = []
    score = []
    for i in range(0, len(writer_genres)):
        writer_name = writer_genres['Writer Name'][i]
        other_genres = writer_genres['Genres'][i]
        writers.append(writer_name)
        genres.append(other_genres)
        score.append(jaccard(my_genres, other_genres))
    df = pd.DataFrame(writers)
    df['genres'] = genres
    df['score'] = score
    df = df[df['score'] != 1.0]
    df = df.sort(['score'], ascending=0)
    
    suggested_genres = df['genres'][0:10]
    
    new_genres = []
    for i in suggested_genres:
        i = i.split(", ")
        for j in i:
            new_genres.append(j)
    new_genres = (set(new_genres)).difference(set(my_genres.split(", ")))
    # if there are no new genres to suggest, suggest the top genres
    if len(new_genres) == 0:
        new_genres = set(["Fantasy", "Young Adult", "Science Fiction"])
    print "I suggest you try writing for the following genres:"
    return new_genres


In [93]:
getSimilar("Nicaless")

I suggest you try writing for the following genres:


{'Romance', 'Science Fiction', 'Young Adult & Youth', 'nan'}

In [94]:
getSimilar("Trillian Anderson")

I suggest you try writing for the following genres:


{'Fanfiction',
 'Non-Fiction',
 'Romance',
 'Science Fiction',
 'Steampunk',
 'Thriller/Suspense',
 'Young Adult',
 'nan'}

In [95]:
getSimilar("AmberMeyer")

I suggest you try writing for the following genres:


{'Fantasy', 'Science Fiction', 'Young Adult'}

In [96]:
getSimilar("Brandon Sanderson")

I suggest you try writing for the following genres:


{'Romance', 'Science Fiction', 'Young Adult & Youth', 'nan'}

Cool! Looks like I have a lot in common with what Brandon Sanderson writes based on our recommendations! 

Of course, this recommender only works for writers already in my list of writers and their known past-written genres, but I'm hoping it's a list that will continue to expand.